In [2]:
pip install nltk

  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [25]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [26]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [27]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [28]:
classes = sorted(list(set(classes)))

In [29]:
print(len(documents), "documents")

104 documents


In [30]:
print(len(classes), "classes",classes)

37 classes ['God', 'abbr', 'about me', 'age', 'alive', 'artificial', 'bookings', 'breathe', 'clone', 'comp', 'computer', 'control', 'creator', 'goodbye', 'greeting', 'hobby', 'imortal', 'joke', 'killing', 'lang', 'lie', 'machine', 'malfunction', 'move', 'myself', 'name', 'options', 'os', 'program', 'sense', 'stories', 'stupid', 'talk', 'tasks', 'thanks', 'usage', 'weather']


In [31]:
print(len(words), "unique lemmatized words", words)

144 unique lemmatized words ["'s", ',', 'a', 'ability', 'about', 'age', 'alive', 'all', 'allowed', 'am', 'an', 'anyone', 'anything', 'are', 'artificial', 'at', 'awesome', 'be', 'being', 'book', 'breathe', 'bye', 'cab', 'call', 'can', 'chatting', 'clone', 'computer', 'control', 'could', 'created', 'creator', 'daddy', 'day', 'detail', 'do', 'doe', 'earth', 'entity', 'everyone', 'exists', 'fake', 'father', 'favorite', 'feature', 'for', 'funny', 'gaurav', 'get', 'god', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hobby', 'hola', 'hotel', 'how', 'i', 'identify', 'immortal', 'in', 'is', 'it', 'joke', 'kill', 'kind', 'know', 'language', 'later', 'laugh', 'lie', 'like', 'linguistic', 'make', 'making', 'malfunction', 'me', 'move', 'murder', 'myself', 'name', 'news', 'next', 'nice', 'not', 'of', 'offered', 'old', 'on', 'operating', 'pointer', 'product', 'profile', 'program', 'provide', 'reservation', 'restraunts', 'robot', 'run', 'sarkar', 'science', 'see', 'sense', 's

In [32]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [34]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [36]:
random.shuffle(training)
training = np.array(training)
x_train = list(training[:,0])
y_train = list(training[:,1])
print("Training Set Created")

Training Set Created


In [38]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [39]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [40]:
cb = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
104/104 [==============================] - 2s 16ms/step - loss: 3.5591 - accuracy: 0.0481
Epoch 2/200
104/104 [==============================] - 0s 719us/step - loss: 3.3720 - accuracy: 0.1250
Epoch 3/200
104/104 [==============================] - 0s 460us/step - loss: 3.1817 - accuracy: 0.2212
Epoch 4/200
104/104 [==============================] - 0s 412us/step - loss: 3.0066 - accuracy: 0.2404
Epoch 5/200
104/104 [==============================] - 0s 441us/step - loss: 2.7557 - accuracy: 0.2885
Epoch 6/200
104/104 [==============================] - 0s 432us/step - loss: 2.4401 - accuracy: 0.4135
Epoch 7/200
104/104 [==============================] - 0s 441us/step - loss: 2.3164 - accuracy: 0.4231
Epoch 8/200
104/104 [==============================] - 0s 451us/step - loss: 1.9573 - accuracy: 0.5385
Epoch 9/200
104/104 [==============================] - 0s 441us/step - loss: 1.9419 - accuracy: 0.4712
Epoch 10/200
104/104 [==============================] - 0s 441us/step - lo

104/104 [==============================] - 0s 460us/step - loss: 0.1830 - accuracy: 0.9519
Epoch 80/200
104/104 [==============================] - 0s 432us/step - loss: 0.1535 - accuracy: 0.9423
Epoch 81/200
104/104 [==============================] - 0s 432us/step - loss: 0.1435 - accuracy: 0.9615
Epoch 82/200
104/104 [==============================] - 0s 451us/step - loss: 0.1353 - accuracy: 0.9615
Epoch 83/200
104/104 [==============================] - 0s 422us/step - loss: 0.1679 - accuracy: 0.9615
Epoch 84/200
104/104 [==============================] - 0s 432us/step - loss: 0.1828 - accuracy: 0.9615
Epoch 85/200
104/104 [==============================] - 0s 403us/step - loss: 0.1391 - accuracy: 0.9519
Epoch 86/200
104/104 [==============================] - 0s 422us/step - loss: 0.1187 - accuracy: 0.9712
Epoch 87/200
104/104 [==============================] - 0s 690us/step - loss: 0.1627 - accuracy: 0.9615
Epoch 88/200
104/104 [==============================] - 0s 470us/step - loss:

104/104 [==============================] - 0s 729us/step - loss: 0.0793 - accuracy: 0.9808
Epoch 158/200
104/104 [==============================] - 0s 479us/step - loss: 0.0668 - accuracy: 0.9712
Epoch 159/200
104/104 [==============================] - 0s 537us/step - loss: 0.1235 - accuracy: 0.9615
Epoch 160/200
104/104 [==============================] - 0s 451us/step - loss: 0.0818 - accuracy: 0.9712
Epoch 161/200
104/104 [==============================] - 0s 460us/step - loss: 0.1203 - accuracy: 0.9423
Epoch 162/200
104/104 [==============================] - 0s 451us/step - loss: 0.1714 - accuracy: 0.9519
Epoch 163/200
104/104 [==============================] - 0s 460us/step - loss: 0.0963 - accuracy: 0.9519
Epoch 164/200
104/104 [==============================] - 0s 451us/step - loss: 0.1119 - accuracy: 0.9423
Epoch 165/200
104/104 [==============================] - 0s 460us/step - loss: 0.1033 - accuracy: 0.9712
Epoch 166/200
104/104 [==============================] - 0s 460us/ste

In [41]:
model.save('chatbot_model.h5')
print("model saved")

model saved
